In [41]:
# import needed libraries
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook


In [42]:
def clean_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('$', 'usd').str.replace('/', 'per').str.replace('&', 'and').str.replace('.', '')
    return df

In [43]:
def clean_values(df):
    # Apply cleaning to all string columns
    for col in df.columns:
        if df[col].dtype == 'object':  # Only process string columns
            df[col] = df[col].astype(str).str.lower().str.replace(' ', '_')
    return df

In [44]:
df_input = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='Measure_List')
clean_column_names(df_input)
clean_values(df_input)

,measure_name,market,fuel,efficiency_levels,sector,competition_group,sub_group
0,furnace,ret_er,oil,1,residential,heating_cooling,oil_furnace
1,furnace,ret_er,natural_gas,1,residential,heating_cooling,gas_furnace
2,central_ac,md,electric,1,residential,heating_cooling,cooling
3,central_ac,ret_er,electric,1,residential,heating_cooling,cooling
4,room_ac,md,electric,1,residential,heating_cooling,cooling
5,room_ac,ret_add_on,electric,1,residential,heating_cooling,cooling
6,fridge,md,electric,2,residential,refrigeration,full_size
7,fridge,ret_er,electric,2,residential,refrigeration,full_size


In [45]:
df_global = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='Global_Inputs')
clean_column_names(df_global)
clean_values(df_global)

# Extract length of analysis value
length_of_analysis = int(df_global.loc[df_global['screening_data'] == 'length_of_analysis', 'info'].values[0])
first_year_of_analysis = int(df_global.loc[df_global['screening_data'] == 'first_year_of_analysis', 'info'].values[0])

# Create a list of years from first_year_of_analysis to first_year_of_analysis + length_of_analysis
years_list = list(range(first_year_of_analysis, first_year_of_analysis + length_of_analysis))
years_list


[2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035]

In [46]:
# this code takes in the input sheet and makes the needed lists


# Measure list
# The input df_input has columns measure_name, market, fuel, sector, efficiency_levels
#The column efficiency_level will be a numeric value representing the number of efficiency levels for that measure
# this code will create a new row for each efficiency level for each measure and add a new column for efficiency level with the value being the level number

rows_list = []
for index, row in df_input.iterrows():
    for level in range(1, row['efficiency_levels'] + 1):
        new_row = row.copy()
        new_row['efficiency_level'] = str(level)
        rows_list.append(new_row)

df_expanded = pd.DataFrame(rows_list)

# Expand market column: if market is "md", create 3 rows with markets "rob", "nc", "reno"
df_expanded = df_expanded.copy()
rows_list_market = []
for index, row in df_expanded.iterrows():
    if row['market'] == 'md':
        # Create three rows for md markets
        for market_type in ['rob', 'nc', 'reno']:
            new_row = row.copy()
            new_row['market'] = market_type
            rows_list_market.append(new_row)
    else:
        # Keep the row as is
        rows_list_market.append(row)

df_expanded = pd.DataFrame(rows_list_market)

# # now if the market column has ret_er make two rows with a new column stock_level and in one row the value is baseline and the other value is existing
rows_list_stock = []
for index, row in df_expanded.iterrows():
    if row['market'] in ['ret_er', 'ret_add_on']:
        # Keep the original row with blank stock_level
        new_row = row.copy()
        new_row['stock_level'] = ''
        rows_list_stock.append(new_row)
        # Create two additional rows for ret_er markets with stock_level
        for stock in ['baseline', 'existing']:
            new_row = row.copy()
            new_row['stock_level'] = stock
            rows_list_stock.append(new_row)
    else:
        # Keep the row as is, but add empty stock_level column for consistency
        new_row = row.copy()
        new_row['stock_level'] = ''
        rows_list_stock.append(new_row)

df_expanded = pd.DataFrame(rows_list_stock)

# For nc, reno, or rob markets with efficiency_level = 1, create baseline row
rows_list_baseline = []
for index, row in df_expanded.iterrows():
    # Keep the original row
    rows_list_baseline.append(row.copy())
    
    # Check if market is nc, reno, or rob AND efficiency_level is 1
    if row['market'] in ['nc', 'reno', 'rob'] and row['efficiency_level'] == '1':
        # Create a baseline row
        baseline_row = row.copy()
        baseline_row['efficiency_level'] = '0'
        baseline_row['stock_level'] = 'baseline'
        rows_list_baseline.append(baseline_row)

df_expanded = pd.DataFrame(rows_list_baseline)

# Now adjust efficiency_level based on stock_level
df_expanded.loc[df_expanded['stock_level'] == 'baseline', 'efficiency_level'] = '0'
df_expanded.loc[df_expanded['stock_level'] == 'existing', 'efficiency_level'] = '-1'

df_expanded.loc[df_expanded['efficiency_level'] == '1', 'stock_level'] = 'efficient'
df_expanded.loc[df_expanded['efficiency_level'] == '2', 'stock_level'] = 'top10'

# I want to concatenate these columns to create a unique measure name
df_expanded['unique_measure_name'] = df_expanded['measure_name'] + "_" + df_expanded['stock_level'] + "_" + df_expanded['fuel'] + "_" + df_expanded['sector']

# Create unique competition_group and subgroup combinations
df_expanded['competition_subgroup'] = df_expanded['competition_group'] + "_" + df_expanded['sub_group']
df_expanded['measure_name_competition_subgroup'] = df_expanded['unique_measure_name'] + "_" + df_expanded['competition_group'] + "_" + df_expanded['sub_group']

#releveant lists
measure_list = df_expanded['unique_measure_name'].unique().tolist()
competition_subgroup_list = df_expanded['competition_subgroup'].unique().tolist()
competition_group = df_expanded['competition_group'].unique().tolist()
subgroup = df_expanded['sub_group'].unique().tolist()
measure_name_competition_subgroup_list = df_expanded['measure_name_competition_subgroup'].unique().tolist() # I dont think the unique removes anything and hopefully it doesnt
sector_list = df_expanded['sector'].unique().tolist()
fuel_list = df_expanded['fuel'].unique().tolist()


df_expanded

,measure_name,market,fuel,efficiency_levels,sector,competition_group,sub_group,efficiency_level,stock_level,unique_measure_name,competition_subgroup,measure_name_competition_subgroup
0,furnace,ret_er,oil,1,residential,heating_cooling,oil_furnace,1,efficient,furnace_efficient_oil_residential,heating_cooling_oil_furnace,furnace_efficient_oil_residential_heating_cool...
0,furnace,ret_er,oil,1,residential,heating_cooling,oil_furnace,0,baseline,furnace_baseline_oil_residential,heating_cooling_oil_furnace,furnace_baseline_oil_residential_heating_cooli...
0,furnace,ret_er,oil,1,residential,heating_cooling,oil_furnace,-1,existing,furnace_existing_oil_residential,heating_cooling_oil_furnace,furnace_existing_oil_residential_heating_cooli...
1,furnace,ret_er,natural_gas,1,residential,heating_cooling,gas_furnace,1,efficient,furnace_efficient_natural_gas_residential,heating_cooling_gas_furnace,furnace_efficient_natural_gas_residential_heat...
1,furnace,ret_er,natural_gas,1,residential,heating_cooling,gas_furnace,0,baseline,furnace_baseline_natural_gas_residential,heating_cooling_gas_furnace,furnace_baseline_natural_gas_residential_heati...
1,furnace,ret_er,natural_gas,1,residential,heating_cooling,gas_furnace,-1,existing,furnace_existing_natural_gas_residential,heating_cooling_gas_furnace,furnace_existing_natural_gas_residential_heati...
2,central_ac,rob,electric,1,residential,heating_cooling,cooling,1,efficient,central_ac_efficient_electric_residential,heating_cooling_cooling,central_ac_efficient_electric_residential_heat...
2,central_ac,rob,electric,1,residential,heating_cooling,cooling,0,baseline,central_ac_baseline_electric_residential,heating_cooling_cooling,central_ac_baseline_electric_residential_heati...
2,central_ac,nc,electric,1,residential,heating_cooling,cooling,1,efficient,central_ac_efficient_electric_residential,heating_cooling_cooling,central_ac_efficient_electric_residential_heat...
2,central_ac,nc,electric,1,residential,heating_cooling,cooling,0,baseline,central_ac_baseline_electric_residential,heating_cooling_cooling,central_ac_baseline_electric_residential_heati...


In [47]:
# this code takes the measure list and makes the workpapers input excel workbook
# the workpapers workbook has a sheet for each measure in the measure list

# Create workbook
wb = Workbook()
# Remove default sheet
if 'Sheet' in wb.sheetnames:
    wb.remove(wb['Sheet'])

# Define all field names
field_names = [
    'parent_condition_name',
    'condition_name',
    'primary_fuel',
    'efficiency_description',
    'efficiency_level',
    'year_start_baseline',
    'year_end_baseline',
    'sector',
    'unit_of_characterization',
    'competition_group',
    'heating_or_cooling',
    'subgroup',
    'base_building_type',
    'interaction_group',
    'equipment_cost',
    'equipment_cost_scale_procedure',
    'labor_cost_installation',
    'effective_useful_life_yrs',
    'water_usage_gal',
    'o_and_m_costs',
    'secondary_hvac_heating_effect_flag',
    'secondary_hvac_cooling_effect_flag',
    'climate_zone_effect',
    'RET_retirement_rate',
    'RET_add_on_applicable',
    'RET_ER_applicable',
    'NC_applicable',
    'ROB_applicable',
    'RENO_applicable',
    'equipment_savings_scale_procedure',
    'fuel-end_use_combinations',
    'fuel_01_type',
    'fuel_01_consumption',
    'fuel_01_units',
    'fuel_01_end_use',
    'fuel_02_type',
    'fuel_02_consumption',
    'fuel_02_units',
    'fuel_02_end_use',
    'fuel_03_type',
    'fuel_03_consumption',
    'fuel_03_units',
    'fuel_03_end_use',
    'fuel_04_type',
    'fuel_04_consumption',
    'fuel_04_units',
    'fuel_04_end_use',
    'fuel_05_type',
    'fuel_05_consumption',
    'fuel_05_units',
    'fuel_05_end_use',
    'fuel_06_type',
    'fuel_06_consumption',
    'fuel_06_units',
    'fuel_06_end_use',
    'fuel_07_type',
    'fuel_07_consumption',
    'fuel_07_units',
    'fuel_07_end_use',
    'fuel_08_type',
    'fuel_08_consumption',
    'fuel_08_units',
    'fuel_08_end_use',
    'fuel_09_type',
    'fuel_09_consumption',
    'fuel_09_units',
    'fuel_09_end_use',
    'fuel_10_type',
    'fuel_10_consumption',
    'fuel_10_units',
    'fuel_10_end_use',
    'fuel_11_type',
    'fuel_11_consumption',
    'fuel_11_units',
    'fuel_11_end_use',
    'fuel_12_type',
    'fuel_12_consumption',
    'fuel_12_units',
    'fuel_12_end_use',
    'fuel_13_type',
    'fuel_13_consumption',
    'fuel_13_units',
    'fuel_13_end_use',
    'fuel_14_type',
    'fuel_14_consumption',
    'fuel_14_units',
    'fuel_14_end_use',
    'fuel_15_type',
    'fuel_15_consumption',
    'fuel_15_units',
    'fuel_15_end_use',
    'fuel_16_type',
    'fuel_16_consumption',
    'fuel_16_units',
    'fuel_16_end_use',
    'fuel_17_type',
    'fuel_17_consumption',
    'fuel_17_units',
    'fuel_17_end_use'
]

# Create a sheet for each measure
for measure in measure_list:
    ws = wb.create_sheet(title=measure)
    # Add header
    ws.append(['Field'])
    # Add all field names as rows in column A
    for field in field_names:
        ws.append([field])

# Save the workbook
wb.save('./00_output/workpapers.xlsx')

c:\Users\AndrewJ.Johnson\OneDrive - NV5\Documents\GitHub\Opt_Energy_Helpers\.conda\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [48]:
# Building type list
df_building_types = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='Building_Types')
clean_column_names(df_building_types)
clean_values(df_building_types)
building_type_list = df_building_types['building_type'].unique().tolist()

In [49]:
#kwh-kw ratio table input sheet
# this code creates a excel workbook with a sheet for kwh-kw ratio table
# the kwh-kw table has columns parent_condition_name, enduse, and the list of building_type
# the first column parent_condition_name is populated with unique measure names from the measure list 
parent_measure_list = df_expanded['measure_name'].unique().tolist()

wb_kwh_kw = Workbook()
# Remove default sheet
if 'Sheet' in wb_kwh_kw.sheetnames:
    wb_kwh_kw.remove(wb_kwh_kw['Sheet'])

# Create the kwh-kw ratio sheet
ws_kwh_kw = wb_kwh_kw.create_sheet(title='kwh_kw_ratio')

# Create the header row with parent_condition_name, enduse, and all building types
headers = ['parent_condition_name', 'enduse'] + building_type_list
ws_kwh_kw.append(headers)

# Populate the parent_condition_name column with unique measure names
for measure_name in parent_measure_list:
    ws_kwh_kw.append([measure_name])

# Save the workbook
wb_kwh_kw.save('./00_output/kwh_kw_ratio.xlsx')

In [50]:
# this code creates the 02_market_characterization input excel workbook
# this workbook has three sheets 
# sheet one is customer_building_type_count which has columns Electric_utility, gas_utility, and every building type from building_type_list
# the values in Electric_utility, gas_utility, are populated from the input sheet Utility_Combinations which has columns electric_utility and gas_utility
# sheet two is equipment_count_per_building_type which has columns competition_group_subgroup populated from competition_subgroup_list and building_type populated from building_type_list
# sheet three is called efficiency_level_breakout and has a column called measure_name_competition_subgroup populated with measure_name_competition_subgroup_list and blank columns with building_type_list

# Read utility combinations from input
df_utilities = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='Utility_Combinations')
clean_column_names(df_utilities)
clean_values(df_utilities)

# Create workbook
wb_market_char = Workbook()
# Remove default sheet
if 'Sheet' in wb_market_char.sheetnames:
    wb_market_char.remove(wb_market_char['Sheet'])

# Sheet 1: customer_building_type_count
ws_customer = wb_market_char.create_sheet(title='customer_building_type_count')
# Create headers
customer_headers = ['electric_utility', 'gas_utility'] + building_type_list
ws_customer.append(customer_headers)
# Populate with utility combinations
for index, row in df_utilities.iterrows():
    ws_customer.append([row['electric_utility'], row['gas_utility']])

# Sheet 2: equipment_count_per_building_type
ws_equipment = wb_market_char.create_sheet(title='equipment_count_per_building_type')
# Create headers
equipment_headers = ['competition_group_subgroup'] + building_type_list
ws_equipment.append(equipment_headers)
# Populate with competition_subgroup values
for comp_subgroup in competition_subgroup_list:
    ws_equipment.append([comp_subgroup])

# Sheet 3: efficiency_level_breakout
ws_efficiency = wb_market_char.create_sheet(title='efficiency_level_breakout')
# Create headers
efficiency_headers = ['measure_name_competition_subgroup'] + building_type_list
ws_efficiency.append(efficiency_headers)
# Populate with measure_name_competition_subgroup values
for measure_comp in measure_name_competition_subgroup_list:
    ws_efficiency.append([measure_comp])

# Save the workbook
wb_market_char.save('./00_output/02_market_characterization.xlsx')

c:\Users\AndrewJ.Johnson\OneDrive - NV5\Documents\GitHub\Opt_Energy_Helpers\.conda\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [51]:
df_ghgs = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='ghgs')
clean_column_names(df_ghgs)
clean_values(df_ghgs)
df_periods = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='Periods')
clean_column_names(df_periods)
clean_values(df_periods)

,periods
0,summer_on-peak
1,summer_off-peak
2,winter_on-peak
3,winter_off-peak
4,shoulder_on-peak
5,shoulder_off-peak
6,summer_gener._capacity
7,winter_gener_capacity
8,summer_td
9,winter_td


In [52]:
ghgs_list = df_ghgs['ghgs'].unique().tolist()
periods_list = df_periods['periods'].unique().tolist()
sector_list = df_expanded['sector'].unique().tolist()
fuel_list = df_expanded['fuel'].unique().tolist()
years_list

[2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035]

In [53]:
# Avoided Costs input workbook creation code will go here
# lists needed period, sectors, fuels, ghgs
# first column is year and populated with years_list the other columns have no values but the headers are each period from periods_list concat with "usdperKWH" 
# the other columns are each fuel type from fuel_list except "electric" concat with "usdperMMBtu" and sector from sector_list all unique combinations

# Create workbook
wb_avoided_costs = Workbook()
# Remove default sheet
if 'Sheet' in wb_avoided_costs.sheetnames:
    wb_avoided_costs.remove(wb_avoided_costs['Sheet'])

# Create the avoided costs sheet
ws_avoided_costs = wb_avoided_costs.create_sheet(title='avoided_costs')

# Build headers
# Start with 'year'
headers = ['year']

# Add period headers with "usdperKWH"
for period in periods_list:
    headers.append(f"{period}_usdperKWH")

# Add fuel/sector combination headers (excluding electric) with "usdperMMBtu"
non_electric_fuels = [fuel for fuel in fuel_list if fuel != 'electric']
for fuel in non_electric_fuels:
    for sector in sector_list:
        headers.append(f"{fuel}_{sector}_usdperMMBtu")

# Append headers to sheet
ws_avoided_costs.append(headers)

# Populate year column with years_list
for year in years_list:
    ws_avoided_costs.append([year])

# Save the workbook
wb_avoided_costs.save('./00_output/11_avoided_costs.xlsx')

In [54]:
# this code is the same as above but every column except year is appended to add a value from the ghgs_list 
# so that we have the same columns but now multiplied by the number of entries in the ghgs_list 
# and the ghgs_list value concat _ to the end of each column name
wb_avoided_costs_ghg = Workbook()
# Remove default sheet
if 'Sheet' in wb_avoided_costs_ghg.sheetnames:
    wb_avoided_costs_ghg.remove(wb_avoided_costs_ghg['Sheet'])

# Create the avoided costs sheet
ws_avoided_costs_ghg = wb_avoided_costs_ghg.create_sheet(title='avoided_costs_ghg')

# Build headers
# Start with 'year'
headers = ['year']

# For each GHG, add all the period and fuel/sector columns
for ghg in ghgs_list:
    # Add period headers with "usdperKWH" and ghg suffix
    for period in periods_list:
        headers.append(f"{period}_usdperKWH_{ghg}")
    
    # Add fuel/sector combination headers (excluding electric) with "usdperMMBtu" and ghg suffix
    non_electric_fuels = [fuel for fuel in fuel_list if fuel != 'electric']
    for fuel in non_electric_fuels:
        for sector in sector_list:
            headers.append(f"{fuel}_{sector}_usdperMMBtu_{ghg}")

# Append headers to sheet
ws_avoided_costs_ghg.append(headers)

# Populate year column with years_list
for year in years_list:
    ws_avoided_costs_ghg.append([year])

# Save the workbook
wb_avoided_costs_ghg.save('./00_output/12_avoided_costs_ghg.xlsx')

In [55]:
# this cell creates the loadshape input excel workbook
# the loadshape workbook has columns unique_measure_name, and periods_list
#the values are from the unique_measure_name column

# Create workbook
wb_loadshape = Workbook()
# Remove default sheet
if 'Sheet' in wb_loadshape.sheetnames:
    wb_loadshape.remove(wb_loadshape['Sheet'])

# Create the loadshape sheet
ws_loadshape = wb_loadshape.create_sheet(title='loadshape')

# Build headers: unique_measure_name followed by all periods
headers = ['unique_measure_name'] + periods_list
ws_loadshape.append(headers)

# Populate unique_measure_name column with values from measure_list
for measure in measure_list:
    ws_loadshape.append([measure])

# Save the workbook
wb_loadshape.save('./00_output/12_loadshape.xlsx')

In [59]:
# incentives input workbook creation code will go here
# the incentives workbook has columns unique_measure_name, utility, electric_incentive_percentage, gas_incentive_percentage and nonutility_incentive_percentage
# the values in unique_measure_name are from measure_list
# the utility column is populated with unique utilities from the Utility_Combinations input sheet
# df_utilities has columns electric_utility and gas_utility they need to be combined into one utility column and made unique
# the measure_list is repeated for each utility with blank values for the incentive percentages

# Combine electric and gas utilities into one unique list
utility_list = pd.concat([df_utilities['electric_utility'], df_utilities['gas_utility']]).unique().tolist()

# Create workbook
wb_incentives = Workbook()
# Remove default sheet
if 'Sheet' in wb_incentives.sheetnames:
    wb_incentives.remove(wb_incentives['Sheet'])

# Create the incentives sheet
ws_incentives = wb_incentives.create_sheet(title='incentives')

# Build headers
headers = ['unique_measure_name', 'utility', 'electric_incentive_percentage', 'gas_incentive_percentage', 'nonutility_incentive_percentage']
ws_incentives.append(headers)

# Populate with measure_list repeated for each utility
for utility in utility_list:
    for measure in measure_list:
        ws_incentives.append([measure, utility])

# Save the workbook
wb_incentives.save('./00_output/13_incentives.xlsx')

In [62]:
# programs this code creates the programs input excel workbook
# the programs workbook has columns program, utility, fuel
# the values in utility are populated from the utility_list
# the values in fuel are populated from the fuel_list
# the values in the column are from programs_list
# there is a unique row for each combination of program, utility, and fuel
# there is a blank column called "non-incentive costs as a percent of incentive costs"
df_programs = pd.read_excel('./00_input/00_Potential_Study_Input_Template.xlsx', sheet_name='Programs')
clean_column_names(df_programs)
clean_values(df_programs)
programs_list = df_programs['program'].unique().tolist()

# Create workbook
wb_programs = Workbook()
# Remove default sheet
if 'Sheet' in wb_programs.sheetnames:
    wb_programs.remove(wb_programs['Sheet'])

# Create the programs sheet
ws_programs = wb_programs.create_sheet(title='programs')

# Build headers
headers = ['program', 'utility', 'fuel', 'non-incentive_costs_as_a_percent_of_incentive_costs']
ws_programs.append(headers)

# Populate with unique combinations of program, utility, and fuel
for program in programs_list:
    for utility in utility_list:
        for fuel in fuel_list:
            ws_programs.append([program, utility, fuel])

# Save the workbook
wb_programs.save('./00_output/14_programs.xlsx')

In [63]:
# this cell creates the 15_line_losses workbook
# the columns are sectors with values from sector_list and periods witch are column names found in periods_list

# Create workbook
wb_line_losses = Workbook()
# Remove default sheet
if 'Sheet' in wb_line_losses.sheetnames:
    wb_line_losses.remove(wb_line_losses['Sheet'])

# Create the line losses sheet
ws_line_losses = wb_line_losses.create_sheet(title='line_losses')

# Build headers: sector followed by all periods
headers = ['sector'] + periods_list
ws_line_losses.append(headers)

# Populate sector column with values from sector_list
for sector in sector_list:
    ws_line_losses.append([sector])

# Save the workbook
wb_line_losses.save('./00_output/15_line_losses.xlsx')

In [64]:
# this cell create the 16_measnonresource workbook 
# the columns are utility and other_nonresource_benefit 
# utility is populated with utility_list and other_nonresource_benefit is blank

# Create workbook
wb_nonresource = Workbook()
# Remove default sheet
if 'Sheet' in wb_nonresource.sheetnames:
    wb_nonresource.remove(wb_nonresource['Sheet'])

# Create the nonresource sheet
ws_nonresource = wb_nonresource.create_sheet(title='nonresource')

# Build headers
headers = ['utility', 'other_nonresource_benefit']
ws_nonresource.append(headers)

# Populate utility column with values from utility_list
for utility in utility_list:
    ws_nonresource.append([utility])

# Save the workbook
wb_nonresource.save('./00_output/16_nonresource.xlsx')

In [65]:
# this cell creates the 17_retail_rates workbook
# the columns are year, utility and a combination of fuel and sector from fuel_list and sector_list
# the year column is populated with years_list
# the utility column is populated with utility_list
# the fuel and sector combination columns are blank but the column names are each fuel from fuel_list concat with each sector from sector_list and usdperKWH for electric fuel and usdperMMBtu for other fuels

# Create workbook
wb_retail_rates = Workbook()
# Remove default sheet
if 'Sheet' in wb_retail_rates.sheetnames:
    wb_retail_rates.remove(wb_retail_rates['Sheet'])

# Create the retail rates sheet
ws_retail_rates = wb_retail_rates.create_sheet(title='retail_rates')

# Build headers
headers = ['year', 'utility']

# Add fuel/sector combination headers
for fuel in fuel_list:
    for sector in sector_list:
        if fuel == 'electric':
            headers.append(f"{fuel}_{sector}_usdperKWH")
        else:
            headers.append(f"{fuel}_{sector}_usdperMMBtu")

ws_retail_rates.append(headers)

# Populate with year and utility combinations
for year in years_list:
    for utility in utility_list:
        ws_retail_rates.append([year, utility])

# Save the workbook
wb_retail_rates.save('./00_output/17_retail_rates.xlsx')

In [66]:
# this cell creates the 20_alternative_competition_ratios input workbook
# the columns are unique_measure_name from measure_list and value which is blank

# Create workbook
wb_alt_comp = Workbook()
# Remove default sheet
if 'Sheet' in wb_alt_comp.sheetnames:
    wb_alt_comp.remove(wb_alt_comp['Sheet'])

# Create the alternative competition ratios sheet
ws_alt_comp = wb_alt_comp.create_sheet(title='alternative_competition_ratios')

# Build headers
headers = ['unique_measure_name', 'value']
ws_alt_comp.append(headers)

# Populate unique_measure_name column with values from measure_list
for measure in measure_list:
    ws_alt_comp.append([measure])

# Save the workbook
wb_alt_comp.save('./00_output/20_alternative_competition_ratios.xlsx')

In [67]:
# this cell creates 30_Construction_Inputs
# the workbook has three sheets called New_Construction, Demolition, Renovation
# all sheets are structured the same
# the columns are competition_subgroup populated from competition_subgroup_list 
# and the other columns are from building_type_list

# Create workbook
wb_construction = Workbook()
# Remove default sheet
if 'Sheet' in wb_construction.sheetnames:
    wb_construction.remove(wb_construction['Sheet'])

# Define sheet names
sheet_names = ['New_Construction', 'Demolition', 'Renovation']

# Create each sheet with the same structure
for sheet_name in sheet_names:
    ws = wb_construction.create_sheet(title=sheet_name)
    
    # Build headers
    headers = ['competition_subgroup'] + building_type_list
    ws.append(headers)
    
    # Populate competition_subgroup column
    for comp_subgroup in competition_subgroup_list:
        ws.append([comp_subgroup])

# Save the workbook
wb_construction.save('./00_output/30_Construction_Inputs.xlsx')

In [69]:
# this cell takes all the individually outputted workbooks and puts them into a single excel workbook

import os
from openpyxl import load_workbook
import re

# Create a new master workbook
master_wb = Workbook()
# Remove default sheet
if 'Sheet' in master_wb.sheetnames:
    master_wb.remove(master_wb['Sheet'])

# List of workbook files to combine (excluding workpapers.xlsx as it has too many sheets)
workbook_files = [
    './00_output/kwh_kw_ratio.xlsx',
    './00_output/02_market_characterization.xlsx',
    './00_output/11_avoided_costs.xlsx',
    './00_output/12_avoided_costs_ghg.xlsx', #might need to change
    './00_output/12_loadshape.xlsx',
    './00_output/13_incentives.xlsx',
    './00_output/14_programs.xlsx',
    './00_output/15_line_losses.xlsx',
    './00_output/16_nonresource.xlsx',
    './00_output/17_retail_rates.xlsx',
    './00_output/20_alternative_competition_ratios.xlsx',
    './00_output/30_Construction_Inputs.xlsx'
]

# Read each workbook and copy sheets to master workbook
for file_path in workbook_files:
    if os.path.exists(file_path):
        wb = load_workbook(file_path)
        # Extract number from filename using regex
        filename = os.path.basename(file_path)
        number_match = re.search(r'(\d+)', filename)
        file_number = number_match.group(1) if number_match else ''
        
        for sheet_name in wb.sheetnames:
            source_sheet = wb[sheet_name]
            # Create new sheet name using file number and original sheet name
            if file_number:
                new_sheet_name = f"{file_number}_{sheet_name}"[:31]  # Excel sheet name limit
            else:
                new_sheet_name = sheet_name[:31]
            target_sheet = master_wb.create_sheet(title=new_sheet_name)
            
            # Copy all data from source to target
            for row in source_sheet.iter_rows():
                target_sheet.append([cell.value for cell in row])

# Save the master workbook
master_wb.save('./00_output/00_Master_Input_Template.xlsx')
print("Master workbook created successfully!")


Master workbook created successfully!
